In [1]:
import numpy as np
import  pandas as pd
import tensorflow as tf,keras
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
from sklearn.metrics import classification_report
from plot_keras_history import plot_history
from keras.preprocessing.sequence import TimeseriesGenerator

In [2]:
data=pd.read_csv('10_minutes_final_processed_gsmfloor_data5.csv')

In [3]:
data.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [4]:
data['label']=data['label'].replace(2,1)
data['label'].value_counts()

0.0    50045
1.0     9954
Name: label, dtype: int64

In [5]:
X=data['combined_result']
y=data['label']
time_series=data['t_dist']

In [6]:
# Logic for creating Sequences
sequences=[]
labels=[]
temp=[]
time=[]
time_temp=[]
for index,value in y.items():
    if value==1:
        temp.append(X[index])
        if (index+1)<len(y) and y[index+1]==0:
            sequences.append(temp)
            labels.append(1)
            time_temp.append(time_series[(index+1)-len(temp)])
            time_temp.append(time_series[index])
            time.append(time_temp)
            time_temp=[]
            temp=[]
    if value==0:
        temp.append(X[index])
        if (index+1)<len(y) and y[index+1]==1:
            sequences.append(temp)
            labels.append(0)
            temp=[]

In [7]:
duration=[]
answer=0
for i in  range(len(time)):
    answer=time[i][1]-time[i][0]
    duration.append(answer)
    
    
    

In [10]:
X_train,X_test,y_train,y_test=train_test_split(sequences,labels,test_size=0.2,random_state=42)

In [9]:
len(sequences)

289

In [26]:
count1=0
count0=0
for i in labels:
    if labels[i]==1:
        count1+=1
    if labels[i]==0:
        count0+=1

In [27]:
print(count1)

724


In [11]:
X_train=pad_sequences(X_train,padding='post',dtype=float,maxlen=50)
X_test=pad_sequences(X_test,padding='post',dtype=float,maxlen=50)

In [12]:
y_train=np.array(y_train)
y_test=np.array(y_test)
y_train=y_train.reshape(-1,1)
y_test=y_test.reshape(-1,1)

In [13]:
X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test=X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [14]:
# Attention Mechanism
from keras.layers import *
from keras.models import *
from keras import backend as K
import keras
tf.keras.saving.get_custom_objects().clear()

In [15]:
#@tf.keras.utils.register_keras_serializable(package="Attention")
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1),
                               initializer='he_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1),
                               initializer='zeros', trainable=True)
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context


In [16]:
model=Sequential([
    Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True) ,input_shape=(X_train.shape[1],1)),# 64
    Bidirectional(tf.keras.layers.LSTM(32,return_sequences=True)),
    attention(),
    tf.keras.layers.Dense(1,activation='sigmoid')
],name="Time-Series-Model")
model.summary()

Model: "Time-Series-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 50, 128)          33792     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 50, 64)           41216     
 nal)                                                            
                                                                 
 attention (attention)       (None, 64)                114       
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 75,187
Trainable params: 75,187
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.load_weights('Sequence-model.h5')

In [18]:
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

In [19]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001)#0.0005,0.001
model.compile(optimizer=optimizer, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [20]:
model_history=model.fit(X_train,y_train,epochs=100,batch_size=16,validation_data=(X_test,y_test),callbacks=[early_stopping])# batch_size=32

Epoch 1/100
15/15 [==============================] - 10s 163ms/step - loss: 0.2937 - accuracy: 0.9004 - val_loss: 0.2729 - val_accuracy: 0.9138
Epoch 2/100
15/15 [==============================] - 1s 55ms/step - loss: 0.2855 - accuracy: 0.8961 - val_loss: 0.2700 - val_accuracy: 0.9138
Epoch 3/100
15/15 [==============================] - 1s 51ms/step - loss: 0.2825 - accuracy: 0.9004 - val_loss: 0.2734 - val_accuracy: 0.9138
Epoch 4/100
15/15 [==============================] - 1s 46ms/step - loss: 0.2818 - accuracy: 0.9004 - val_loss: 0.2694 - val_accuracy: 0.9138
Epoch 5/100
15/15 [==============================] - 1s 48ms/step - loss: 0.2873 - accuracy: 0.8961 - val_loss: 0.2651 - val_accuracy: 0.9138
Epoch 6/100
15/15 [==============================] - 1s 47ms/step - loss: 0.2909 - accuracy: 0.8918 - val_loss: 0.2649 - val_accuracy: 0.9138
Epoch 7/100
15/15 [==============================] - 1s 44ms/step - loss: 0.2842 - accuracy: 0.9004 - val_loss: 0.2637 - val_accuracy: 0.9138
Epoc